<a href="https://colab.research.google.com/github/DarkXonline/Torrent-To-GDrive-Download/blob/main/Copy_of_Torrent_To_Google_Drive_Downloader_v4_1%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader v4.1 v2

### Mount Google Drive
To stream files we need to mount Google Drive.

In [ ]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/gdrive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

### Extras

**Google Colab Timeout Preventer! Run this Code to prevent TIMEOUT in Google Colab.Before running any code run this!**

In [ ]:
#@markdown <br><center><img src='https://github.com/DarkXonline/Torrent-To-GDrive-Download/blob/main/imgs/2.png?raw=true' height="100" alt="netdata"/></center>
#@title <center>Time Out Preventer (Advanced) </center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent NOTEBOOK from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

**Check from which speed Google Will transfer your Files: (Optional)**

In [ ]:
#@markdown <br><center><img src='https://github.com/DarkXonline/Torrent-To-GDrive-Download/blob/main/imgs/1.jpg?raw=true' height="50" alt="ST-logo"/></center>
#@markdown <center><h3>Check Transfer Speed!</h3></center><br>
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python

###Dependency
https://www.libtorrent.org/

In [ ]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent && apt install python3-libtorrent

### Code to download torrent

In [ ]:
#@markdown <center><h3>Start Download to Google Drive</h3></center><br>
import libtorrent as lt
import time
import datetime

# User Input
#@markdown Change zipp to "Yes" if you want to zip your file.
zipp = "No" #@param ["No", "Yes"]
link = "\"Input Torrent Link or Magnet here\"" #@param {type:"string"}

#@markdown Custom path inside the /Torrent/ (if needed) [ended with **'/'**] :
Destination_Path = "" #@param {type:"string"}

def download_torrent():
    ses = lt.session()
    ses.listen_on(6881, 6891)
    # link = input("Input Torrent Link or Magnet and Press Enter: ")
    print(link)

    handle = lt.add_magnet_uri(ses, link, params)
    # change the 0 to a 1 to download sequential - this sequential option is only if you selected zip. If not,
    # scroll farther down.
    handle.set_sequential_download(0)

    ses.start_dht()
    begin = time.time()
    print(datetime.datetime.now())

    print('Downloading Metadata...')
    while not handle.has_metadata():
        time.sleep(1)
    print('Got Metadata, Starting Torrent Download...')

    print("Starting", handle.name())

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
              (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
               s.num_peers, state_str[s.state]))
        time.sleep(5)

    end = time.time()
    print(handle.name(), "COMPLETE")

    print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")

    print(datetime.datetime.now())

default_path = '/content/drive/MyDrive/Torrent/'

if Destination_Path != '':
  final_path = default_path + Destination_Path
else:
  final_path = default_path

if zipp == 'Yes' or zipp == 'y':
    #@markdown Enter zip name if zipp set to **"Yes"** :
    zip_name = "\"Enter zip name\"" #@param {type:"string"}
    import shutil

    params = {
        'save_path': '/content/temp/',
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

    print("now time to zip")

    print("Zipping ...")
    final_zip_directory = final_path + zip_name
    shutil.make_archive(final_zip_directory, 'zip', '/content/temp/')
    shutil.rmtree('/content/temp/')

else:
    params = {
        'save_path': final_path,
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

print('\nALL DONE!')
